# Change detection of series of point clouds

In this note book, we apply our methods to a series of circles with increasing radius.

In [ ]:
import gudhi as gd
from gudhi.representations import Landscape
import numpy as np
import matplotlib.pyplot as plt
from functools import partial

In [ ]:
from mdl.model import Norm1D
from mdl.smdl import SMDL
from bocpd.mybocpd import BOCD, StudentT, constant_hazard
from mdl.ppm import get_K_mu_sigma
from mdl.wkc import get_WKC
from mdl.kc import get_KC
from utils.evaluation import calc_auc_average, calc_falarms_benefit, InvRunLen, get_threshold
from utils.embedding import TimeDelayEmbedding

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Generate dataset
We generate a series circles with increasing radius.

In [ ]:
def generate_circles(phases):
    x = []
    y = []
    ks = []
    for i in range(200):
        j = int(i/50)
        np.random.seed(i)
        ks.append(phases[j]+np.random.normal(0,0.001))
    
    for i in range(len(ks)):
        k = ks[i]
        x = []
        y = []
        for j in np.arange(0,2,0.015):
            x.append(k*np.sin(np.pi*j))
            y.append(k*np.cos(np.pi*j))
        mean = np.array([0,0])  
        cov=np.array([[0.10,0.06],[0.06,0.10]])
        np.random.seed(i)
        data_1 = np.random.multivariate_normal(mean, cov, size=len(np.arange(0,2,0.015)))
        x += data_1.T[0]
        y += data_1.T[1]
        t_data = np.array([[x,y]])
        if i == 0:
            data = t_data
        else:
            data = np.append(data,t_data,axis=0)
            
    return data

In [ ]:
phases=[0.4,1.2,2.0,2.8]
data = generate_circles(phases)

In [ ]:
for i in range(len(data)):
    if i % 20 == 0:
        plt.scatter(data[i][0],data[i][1])
        plt.title("t="+str(i))
        plt.xlim(-5,5)
        plt.ylim(-5,5)
        plt.show()

## Number of optimal components in Persistence Parametric Model
We apply the PPM method to the PDs of the point clouds.

In [ ]:
Ks = []
max_K = 5
b = 10
for i in range(len(data)):
    ob_data = data[i]
    alpha_complex = gd.AlphaComplex(points=ob_data.T)
    simplex_tree = alpha_complex.create_simplex_tree()
    diag = simplex_tree.persistence()
    A = simplex_tree.persistence_intervals_in_dimension(1)
    K, mu, sigma = get_K_mu_sigma(A, max_K, b)
    Ks.append(K)

In [ ]:
plt.plot(Ks)
plt.title("The number of mixture")
plt.show()

We plot the centers of PPM at several time.

In [ ]:
for i in [60,120,180]:
    ob_data = data[i]
    alpha_complex = gd.AlphaComplex(points=ob_data.T)
    simplex_tree = alpha_complex.create_simplex_tree()
    diag = simplex_tree.persistence()
    A = simplex_tree.persistence_intervals_in_dimension(1)
    K, mu, sigma = get_K_mu_sigma(A, max_K, b)
    
    plt.xlim(-0.05,1.5)
    plt.ylim(-0.05,5)
    plt.scatter(A.T[0],A.T[1]-A.T[0])
    for l in range(K):
        plt.scatter(mu[l][0],mu[l][1],color="red")
    plt.title("(t="+str(i)+")")
    plt.show()

We smooth the series of the number of mixture components and apply Bayesian online change point detection (BOCPD).

In [ ]:
smooth = 2
smooth_Ks = [0]*(smooth-1)
for i in range(smooth-1,len(Ks)):
    smooth_Ks.append(np.mean(Ks[i-smooth+1:i+1]))

In [ ]:
plt.plot(smooth_Ks)
plt.title("The number of mixture (smoothed)")
plt.show()

In [ ]:
N_trial = 1
ALPHA = 0.1
BETA = 1.0
KAPPA = 1.0
MU = 0.0
DELAY = 15
T = 5

for LAMBDA in [2,5,10,20,40,60,80,100]:
    for THRESHOLD in [0.1, 0.3]:
        scores_bocpd = []
        for i in range(N_trial):
            X = smooth_Ks

            # BOCPD
            bocd = BOCD(partial(constant_hazard, LAMBDA),
                        StudentT(ALPHA, BETA, KAPPA, MU), X)
            change_points = []
            scores = [np.nan] * DELAY
            for x in X[:DELAY]:
                bocd.update(x)
            for x in X[DELAY:]:
                bocd.update(x)
                if bocd.growth_probs[DELAY] >= THRESHOLD:
                    change_points.append(bocd.t - DELAY + 1)
                score = np.sum(bocd.growth_probs[:bocd.t - DELAY] * 1.0 / (1.0 + np.arange(1, bocd.t - DELAY + 1)))
                scores.append(score)

            scores_bocpd.append(scores)

        scores_bocpd = np.array(scores_bocpd)
        auc_list = calc_auc_average(scores_bocpd,np.array([50,100,150]),T=T)
        print('LAMBDA =', LAMBDA, 'THRESHOLD =', THRESHOLD, 'AUC:', np.mean(auc_list))

## Kernel Complexity of Persistence Non-Parametric Model
We apply the PNPM method to the PDs of the point clouds.

In [ ]:
KCs_PNPM = []
epsilon = 0.1
gamma = 0.7
param = 1.0
for i in range(len(data)):
    ob_data = data[i]
    alpha_complex = gd.AlphaComplex(points=ob_data.T)
    simplex_tree = alpha_complex.create_simplex_tree()
    diag = simplex_tree.persistence()
    A = simplex_tree.persistence_intervals_in_dimension(1)
    x1 = np.append(np.array([A.T[0]]),[A.T[1]-A.T[0]],axis=0)
    x = x1.T
    n = len(x)
    m = len(x[0])
    if len(x) > 0:
        KC = get_WKC(x, n, m, gamma, epsilon, param)
        KCs_PNPM.append(KC)
    else:
        KCs_PNPM.append(0)

We apply sequential MDL-change statistics (SMDL) to the series of the kernel complexity of PNPM.

In [ ]:
plt.plot(KCs_PNPM)
plt.title("Kernel Complexity of PNPM")
plt.show()

In [ ]:
h = 10
cps_true = np.array([50, 100,150])
N_trial = 1
mu_max = 50.0
sigma_min = 0.005
T = 5

scores_list_0th = []
scores_list_1st = []
scores_list_2nd = []
for i in range(N_trial):
    X = np.array(KCs_PNPM)
    len_X = len(X)
    
    norm1d = Norm1D()
    smdl = SMDL(norm1d)

    scores_0th = np.array([np.nan]*h + [ smdl.calc_change_score(X[(t-h):(t+h)], h, mu_max=mu_max, sigma_min=sigma_min) \
                                     for t in range(h, len_X-h)] + [np.nan]*h)
    scores_list_0th.append(scores_0th)

scores_list_0th = np.array(scores_list_0th)
auc_list_0th = calc_auc_average(scores_list_0th, cps_true=cps_true,T=T)
print("AUC:", np.mean(auc_list_0th))

## Comparison with existing methods
Below we apply several existing methods to the time-series for comparison.

### L2 norm of persistence landscape

In [ ]:
L2_norms = []
for i in range(len(data)):
    ob_data = data[i]
    alpha_complex = gd.AlphaComplex(points=ob_data.T)
    simplex_tree = alpha_complex.create_simplex_tree()
    simplex_tree.persistence()
    A = simplex_tree.persistence_intervals_in_dimension(1)
    x1 = np.append(np.array([A.T[0]]),[A.T[1]-A.T[0]],axis=0)
    x = x1.T
    LS = Landscape(num_landscapes=3,resolution=1000)
    L = LS.fit_transform([simplex_tree.persistence_intervals_in_dimension(1)])
    L2 = 0
    L2 += pow(np.linalg.norm(L[0][:1000],ord=2),2)
    L2 += pow(np.linalg.norm(L[0][1000:2000],ord=2),2)
    L2 += pow(np.linalg.norm(L[0][2000:3000],ord=2),2)
    L2_norms.append(pow(L2,1/2))

In [ ]:
plt.plot(L2_norms)
plt.title("L2 norm")
plt.show()

In [ ]:
h = 5
cps_true = np.array([50,100,150])
N_trial = 1
mu_max = 50.0
sigma_min = 0.005
T = 5

scores_list_0th = []
scores_list_1st = []
scores_list_2nd = []
for i in range(N_trial):
    X = np.array(L2_norms)
    len_X = len(X)
    
    norm1d = Norm1D()
    smdl = SMDL(norm1d)

    scores_0th = np.array([np.nan]*h + [ smdl.calc_change_score(X[(t-h):(t+h)], h, mu_max=mu_max, sigma_min=sigma_min) \
                                     for t in range(h, len_X-h)] + [np.nan]*h)
    scores_list_0th.append(scores_0th)

scores_list_0th = np.array(scores_list_0th)
auc_list_0th = calc_auc_average(scores_list_0th, cps_true=cps_true,T=T)
print("AUC:", np.mean(auc_list_0th))

### Kernel complexity applied to the original point clouds

In [ ]:
epsilon = 0.1
gamma = 0.7
param = 1.0
KCs = []
for i in range(len(data)):
    ob_data = data[i]
    x = ob_data.T
    n = len(x)
    m = len(x[0])
    if len(x)>0:
        KC = get_KC(x, n, m, gamma, epsilon, param)
        KCs.append(KC)
    else:
        KCs.append(0)

In [ ]:
plt.plot(KCs)
plt.title("Kenel Complexity of the original point clouds")
plt.show()

In [ ]:
h = 10
cps_true = np.array([50, 100,150])
N_trial = 1
mu_max = 50.0
sigma_min = 0.005
T = 5

scores_list_0th = []
scores_list_1st = []
scores_list_2nd = []
for i in range(N_trial):
    X = np.array(KCs)
    len_X = len(X)
    
    norm1d = Norm1D()
    smdl = SMDL(norm1d)

    scores_0th = np.array([np.nan]*h + [ smdl.calc_change_score(X[(t-h):(t+h)], h, mu_max=mu_max, sigma_min=sigma_min) \
                                     for t in range(h, len_X-h)] + [np.nan]*h)
    scores_list_0th.append(scores_0th)

scores_list_0th = np.array(scores_list_0th)
auc_list_0th = calc_auc_average(scores_list_0th, cps_true=cps_true,T=T)
print("AUC:", np.mean(auc_list_0th))